<a href="https://colab.research.google.com/github/Abhinavjha07/Human-Activity-Recognition/blob/master/3DCNN_FeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/Abhinavjha07/ML_Datasets/

Cloning into 'ML_Datasets'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 40603 (delta 4), reused 0 (delta 0), pack-reused 40593
Receiving objects: 100% (40603/40603), 1.35 GiB | 13.46 MiB/s, done.
Resolving deltas: 100% (1255/1255), done.
Checking out files: 100% (40507/40507), done.


In [0]:
import cv2
import numpy as np
import os
from keras.utils import to_categorical
from tqdm import tqdm

directory = '/content/ML_Datasets/UCF_11'

classes = {
    'basketball' : 0,
    'biking' : 1,
    'diving' : 2,
    'golf_swing' : 3,
    'horse_riding' : 4,
    'soccer_juggling' : 5,
    'swing' : 6,
    'tennis_swing' : 7,
    'trampoline_jumping' : 8,
    'volleyball_spiking' : 9,
    'walking' : 10
    }

Y = []
X = []
for c in os.listdir(directory):
    print(c)
    if c != 'readme.txt':
        label = classes[c]
        d = directory + '/'+c
        for x in os.listdir(d):
            if x != 'Annotation':
                z = d+'/'+x
                for file in os.listdir(z):
                    cap = cv2.VideoCapture(z+'/'+file)
                    fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
                    frames = []
                    i = 0
                    while(cap.isOpened()):
                        ret,fr = cap.read()
                        if ret!=True:
                            break

                        fr = cv2.cvtColor(fr,cv2.COLOR_BGR2GRAY)
                        fr = cv2.resize(fr,(64,64),interpolation = cv2.INTER_LINEAR)
                        fr = fgbg.apply(fr)

                        if i%3 == 0:
                            frames.append(np.array(fr))

                        i += 1
                    if len(frames) == 0:
                        print(c,x,file)
                    if len(frames) != 0:
                        X.append(np.array(frames))
                        Y.append(np.array(label))



X = np.array(X)
Y = np.array(Y)


print(X.shape)
print(Y.shape)
train_X = []
for i in range(X.shape[0]):
    print(X[i].shape)
    if X[i].shape[0] < 50:
        temp = np.zeros((50,64,64))
        temp[0:X[i].shape[0],:,:] = X[i][0]
        
    else:
        temp = X[i][0:50,:,:]

    train_X.append(temp)

train_X = np.array(train_X)
print(train_X.shape)
Y = np.reshape(Y,(1652,1))
Y = to_categorical(Y)
print(Y.shape)


np.save('X.npy',train_X)
np.save('Y.npy',Y)
 
   
    


walking
swing
volleyball_spiking
readme.txt
diving
soccer_juggling
biking
basketball
trampoline_jumping
golf_swing
tennis_swing
horse_riding
(1652,)
(1652,)
(41, 64, 64)
(67, 64, 64)
(67, 64, 64)
(38, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(60, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(58, 64, 64)
(57, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(56, 64, 64)
(53, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(51, 64, 64)
(35, 64, 64)
(67, 64, 64)
(67, 64, 64)
(58, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(34, 64, 64)
(34, 64, 64)
(36, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(51, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64)
(67, 64, 64

In [1]:
import keras
from keras.layers import Dense,Dropout,Conv3D,Flatten,MaxPooling3D,Input,Add,Reshape
import numpy as np
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Model


n_classes = 11

X = np.load('X.npy')
Y = np.load('Y.npy')
X = np.reshape(X,(-1,50,64,64,1))

train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size=0.2,random_state=0)
train_X = np.concatenate((train_X,test_X))
train_Y = np.concatenate((train_Y,test_Y))
input_shape = (50,64,64,1)

inp = Input(shape=input_shape)
encode = Conv3D(64,(3,3,3),activation='relu')(inp)
encode = MaxPooling3D((2,2,2),strides = (3,2,2))(encode)
print(encode.shape)
encode = Conv3D(128,(5,5,5),activation = 'relu')(encode)
encode = MaxPooling3D((3,3,3),strides = (3,2,2))(encode)
print(encode.shape)
encode = Conv3D(256,(4,7,7),activation = 'relu')(encode)
print(encode.shape)

encoder = Model(inp,encode)


encode = Flatten()(encode)
print(encode.shape)


x = Dense(128,activation='relu')(encode)
x = Dropout(0.5)(x)
x = Dense(64,activation = 'relu')(x)
x = Dropout(0.4)(x)
x = Dense(n_classes,activation= 'softmax')(x)

model = Model(inputs = inp,outputs = x)
model.summary()
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.fit(train_X,train_Y,epochs = 10,validation_data=(test_X,test_Y))

model.save('3DCNN_UCF11')
encoder.save('encoding')





Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
(?, 16, 31, 31, 64)
(?, 4, 13, 13, 128)
(?, 1, 7, 7, 256)
(?, ?)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 64, 64, 1)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 48, 62, 62, 64)    1792      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 16, 31, 31, 64)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 27, 27, 128)   1024128   
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 4, 13, 13, 128)    0         
______________________

In [3]:
import numpy as np
import keras
from keras.layers import Dense,Dropout,Conv3D,Flatten,MaxPooling3D,Input,Add,Reshape
import numpy as np
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import Model,load_model

encoder = load_model('encoding')
X = np.load('X.npy')
X = np.reshape(X,(-1,50,64,64,1))
encode_X = encoder.predict(X)
print(encode_X.shape)

np.save('encode_X',encode_X)





/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


(1652, 1, 7, 7, 256)
